In [23]:
import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np

In [30]:
model_height = XGBRegressor()
model_height.load_model("model_height.json")

model_period = XGBRegressor()
model_period.load_model("model_period.json")

model_power = XGBRegressor()
model_power.load_model("model_power.json")

In [31]:

# 驗證是否保留特徵名稱
print("模型的特徵名稱:", model_height.get_booster().feature_names)

模型的特徵名稱: ['speed', 'direction', 'pressure', 'temperature', 'power', 'directionType_Offshore', 'directionType_Onshore']


In [26]:
direction_mapping = {
    'Cross-shore': [0, 0],  # Cross-shore 時，兩個特徵都為 0
    'Offshore': [1, 0],     # Offshore 時，Offshore=1, Onshore=0
    'Onshore': [0, 1]       # Onshore 時，Offshore=0, Onshore=1
}

def preprocess_input(input_data):
    """
    將輸入數據進行 one-hot encoding，並返回處理後的數據
    :param input_data: dict, 包含原始輸入數據
    :return: dict, 包含 one-hot 編碼特徵的數據
    """
    # One-hot encoding
    direction_type = input_data['directionType']
    direction_one_hot = direction_mapping.get(direction_type, [0, 0])  # 默認為 [0, 0]

    # 將 one-hot 編碼加入輸入數據
    input_data['directionType_Offshore'] = direction_one_hot[0]
    input_data['directionType_Onshore'] = direction_one_hot[1]

    # 刪除原始的 directionType
    del input_data['directionType']

    return input_data


In [35]:
def predict_surf_conditions(input_data):
    """
    預測浪況並判斷是否適合衝浪
    :param input_data: dict, 包含原始輸入數據
    :return: dict, 包括預測結果和提醒信息
    """
    # 預處理輸入數據
    input_data = preprocess_input(input_data)

    # 特徵提取
    height_features = np.array([[
        input_data['speed'], input_data['direction'], input_data['pressure'],
        input_data['temperature'], input_data['power'],
        input_data['directionType_Offshore'], input_data['directionType_Onshore']
    ]])

    period_features = np.array([[
        input_data['gust'], input_data['surf_min'], input_data['surf_max'],
        input_data['speed'], input_data['direction'], input_data['pressure'],
        input_data['temperature'], input_data['power'],
        input_data['directionType_Offshore'], input_data['directionType_Onshore']
    ]])

    power_features = np.array([[
        input_data['surf_min'], input_data['surf_max'], input_data['speed'],
        input_data['direction'], input_data['pressure'], input_data['temperature'],
        input_data['power'], input_data['directionType_Offshore'], input_data['directionType_Onshore']
    ]])

    # 預測結果
    swell_height = model_height.predict(height_features)[0]
    swell_period = model_period.predict(period_features)[0]
    swell_power = model_power.predict(power_features)[0]


    # 判斷是否適合衝浪
    offshore = input_data['directionType_Offshore']  # 判斷是否為 Offshore 狀態
    suitable_for_surfing = (
        8 <= swell_period <= 16 or  # 週期 8-16 秒
        (6 <= swell_period < 8 and offshore == 1)  # 週期 6-8 秒 且 Offshore
    )

    # 添加提醒信息
    warnings = []
    if input_data['speed'] > 19:
        warnings.append("風大提醒：風速超過 19 m/s")
    if input_data['temperature'] < 20:
        warnings.append("低溫提醒：溫度低於 20°C")

    return {
        'swell_height': swell_height,
        'swell_period': swell_period,
        'swell_power': swell_power,
        'suitable_for_surfing': suitable_for_surfing,
        'warnings': warnings
    }

In [40]:
input_data = {
    'speed': 20.5,               # 風速
    'direction': 270,            # 風向
    'pressure': 1012,            # 氣壓
    'temperature': 18,           # 溫度
    'power': 2000,               # 功率
    'directionType': 'Offshore', # 風向類別
    'gust': 25.0,                # 陣風
    'surf_min': 1.2,             # 最小浪高
    'surf_max': 2.3              # 最大浪高
}

# 預測浪況
result = predict_surf_conditions(input_data)

# 顯示結果
print(f"預測Swell Height: {result['swell_height']:.2f} m")
print(f"預測Swell Period: {result['swell_period']:.2f} s")
print(f"預測Swell Power: {result['swell_power']:.2f}")
print('----------------------------------------------')
if result['suitable_for_surfing']:
    print("適合衝浪！")
else:
    print("不適合衝浪！")
print('----------------------------------------------')
if result['warnings']:
    print("提醒事項：")
    for warning in result['warnings']:
        print(f" - {warning}")

預測Swell Height: 1.11 m
預測Swell Period: 11.55 s
預測Swell Power: 502.56
----------------------------------------------
適合衝浪！
----------------------------------------------
提醒事項：
 - 風大提醒：風速超過 19 m/s
 - 低溫提醒：溫度低於 20°C
